In [ ]:
import pandas as pd
from pathlib import Path
from data.data_utils import (
    get_csv_path,
    get_json_path,
    get_analest_uv_vis_path,
    get_analest_fluorescence_path,
    get_optics_table_path,
    get_plot_path,
)
from data.spectrum import Spectrum
from visualization.plotter import spectrum_subplots, calibration_curve, stacked_spectra
from experiments.csv_to_json import csv_to_json

In [ ]:
path_callables: dict = {
    "optics": {
        "uv_vis": get_optics_table_path,
        "photoluminescence": get_optics_table_path,
    },
    "ANALEST": {
        "uv_vis": get_analest_uv_vis_path,
        "photoluminescence": get_analest_fluorescence_path,
    },
}

spectrum_callables: dict = {
    "optics": {
        "uv_vis": Spectrum.from_optics_table,
        "photoluminescence": Spectrum.from_optics_table,
    },
    "ANALEST": {
        "uv_vis": Spectrum.from_analest_uv_vis,
        "photoluminescence": Spectrum.from_analest_fluorescence,
    },
}

plot_callables: dict = {
    "subplots": spectrum_subplots,
    "calibration": calibration_curve,
    "stacked": stacked_spectra,
}

In [ ]:
# Variables to change for a type of plot (linear calibration, mass spec, subplots) and which set of experiments for metadata.
type_of_plot: str = "subplots"
experiment_set: str = "expt7"

In [ ]:
# Gather data
# load experiments.json that you want to plot
experiments_path: Path = Path.cwd() / "experiments" / "experiments.json"
experiments: dict = pd.read_json(experiments_path)
experiments_to_plot: list[str] = experiments[type_of_plot][experiment_set][
    "experiments"
]
metadata_csv_path: Path = get_csv_path(
    experiments[type_of_plot][experiment_set]["experiment_code"]
)
metadata_json_path: Path = get_json_path(
    experiments[type_of_plot][experiment_set]["experiment_code"]
)
csv_to_json(metadata_csv_path, metadata_json_path)

In [ ]:
# Gather Spectrum into list for plotting
spectra: list = []
# Iterate through types of measurements
for measurement in experiments[type_of_plot][experiment_set]["measurements"]:
    # Iterate through experiments to plot
    for experiment_to_plot in experiments_to_plot:
        # get path for specific experiment
        experiment_to_plot_path: Path = path_callables[
            experiments[type_of_plot][experiment_set]["instrument"]
        ][measurement](experiment_to_plot)
        # create spectrum object with path, etc.
        spectrum: Spectrum = spectrum_callables[
            experiments[type_of_plot][experiment_set]["instrument"]
        ][measurement](
            experiment_to_plot_path,
            experiment_to_plot,
            experiments[type_of_plot][experiment_set]["experiment_code"],
            metadata_json_path,
            measurement,
        )
        spectra.append(spectrum)

In [ ]:
# Sanity Check before plotting
if len(spectra) == 0:
    raise ValueError("No spectra to plot")
if len(spectra[0].wavelengths) == 0:
    raise ValueError("No wavelengths to plot")
if len(spectra[0].intensity) == 0:
    raise ValueError("No intensities to plot")

In [ ]:
# plot the list of spectra
# calibration curves require extra parameter of lambda_max
if type_of_plot == "calibration":
    plot_callables[type_of_plot](
        spectra,
        experiments[type_of_plot][experiment_set]["lambda_max"],
        get_plot_path(
            spectra, experiments[type_of_plot][experiment_set]["experiment_code"]
        ),
    )
else:
    plot_callables[type_of_plot](
        spectra,
        get_plot_path(
            spectra, experiments[type_of_plot][experiment_set]["experiment_code"]
        ),
    )